In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import tensorflow as tf
import pandas as pd
import io
import numpy as np

In [0]:
def cnn_model_fn(features, labels, mode):
  
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  
  conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  
  conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {"classes": tf.argmax(input=logits, axis=1), "probabilities": tf.nn.softmax(logits, name="softmax_tensor")}
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
  
  eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [0]:
def one_hot_fn(values):
  
  ret = []
  for value in values:
    tmp = np.zeros(10)
    tmp[value] = 1
    ret.append(tmp)
  return (np.array(ret,dtype=int))

In [0]:
def threshold_fn(X):
  for i in len(X):
    for j in len(X[i]):
      if X[i][j] >= 100:
        X[i][j] = 1
      else:
        X[i][j] = 0
  return X

In [0]:
train = pd.read_csv(io.StringIO(uploaded['mnist_train.csv'].decode('utf-8')))

train_labels = pd.DataFrame()
train_labels['Category'] = train['Category']
train_values = train.drop(['Id', 'Category'], axis=1)

train_data = np.array(train_values.values.tolist(), dtype=float)
train_lab = np.array(train_labels.values.tolist(), dtype=int)

train_data = threshold_fn(train_data)
train_labe = one_hot_fn(train_lab)

test1 = pd.read_csv(io.StringIO(uploaded['mnist_test.csv'].decode('utf-8')))
test_df = test1.drop(['Id'], axis=1)
test_values1 = np.array(test_df.values.tolist(), dtype=float)
test_values1 = threshold_fn(test_values1)

In [0]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": train_data}, y=train_lab, batch_size=100, num_epochs=None, shuffle=True)

mnist_classifier.train(input_fn=train_input_fn, steps=20000)

In [0]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": test_values1}, shuffle=False)
preds = mnist_classifier.predict(pred_input_fn)

In [0]:
predictions = []

for i in range(10000):
  predictions.append(next(preds)['classes'])

In [0]:
full_mat = []
for i in range(len(predictions)):
  full_mat.append([i+1, predictions[i]])

labels = ['Id', 'Category']
new_df = pd.DataFrame.from_records(full_mat, columns=labels)
new_df.to_csv('predictions_cnn.csv',index=False)
#print(new_df.head(5))
files.download('predictions_cnn.csv')